In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))

In [2]:
import torch
from Data import GraphDataset
from Models import GCNFeatureExtractor
from utils.train import train


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\melan\shared-folder\Thesis\SSL-for-Quantum-Cirucits-Generation\qiskit_env\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\melan\shared-folder\Thesis\SSL-for-Quantum-Cirucits-Generation\qiskit_env\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\melan\shared-folder\Thesis\SSL-fo

In [3]:
use_pre_paired = True
batch_size = 4
lr = 1e-3
epochs = 200
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_size = 50
pooling = 'last_avg'

In [4]:
# load data 
from Data import load_graphs

basic_data, _ = load_graphs()

	Collected 1 sample from null_ops.
	Collected 4 samples from commutations.
	Collected 7 samples from equivalences.
	Collected 7 samples from combined.
Loaded 19 samples and 19 quantum circuits from subset.


In [5]:
# import pickle

# def load_graphs(data_dir='../Data/raw/', file_name='handcrafted_dataset.pkl', subset=None):
#     file_path = os.path.join(data_dir, file_name)
#     graphs, qcs = [], []
    
#     with open(file_path, 'rb') as f:
#         dataset = pickle.load(f)

#     if file_name == 'handcrafted_dataset.pkl':
#         if subset is not None:
#             dataset = dataset[subset]
#             print(f"Loaded {len(dataset)} elements from subset {subset}:")
#         # collect all graphs in dataset, which are stored in a nested dictionary
#         dataset = collect_from_dict(dataset) 

#     # extract graphs and qcs separately, if needed
#     for sample in dataset:
#         if isinstance(sample, tuple): # if a tuple (qc, graph)
#             g, qc = sample
#             graphs.append(g)
#             qcs.append(qc)
#         elif all(isinstance(graph, tuple) for graph in sample):# if a list of tuples (qc, graph) 
#             g, qc = zip(*sample)
#             qcs.append(list(qc))
#             graphs.append(list(g))

#     print(f"Loaded {len(graphs)} samples and {len(qcs)} quantum circuits from subset.")
    
#     return graphs, qcs

# def collect_from_dict(dictionary):
#     graphs = []
#     for k, v in dictionary.items():
#         if isinstance(v, dict):
#             graphs.extend(collect_from_dict(v))
#         elif isinstance(v, list):
#             if all(isinstance(item, list) for item in v):  # if list of lists
#                 graphs.extend(v)
#                 print(f"\tCollected {len(v)} items from {k}.")
#             elif all(isinstance(item, tuple) for item in v):  # if list of tuples
#                 graphs.append(v)
#                 print(f"\tCollected 1 sample from {k}.")
                
#     return graphs

## Basic transforms training

In [6]:
dataset = GraphDataset(basic_data, pre_paired=use_pre_paired)

In [7]:
graphs = basic_data[0]
print(len(graphs))

18


In [8]:
import numpy as np
import networkx as nx
def get_attr_matrix(graph):
    nodes_list = list(graph.nodes)
    nodes_view = graph.nodes(data=True)
    return np.array([nodes_view[node]['feature_vector'] for node in nodes_list])

In [9]:
# Check if graphs in the dataset are correctly converted to PyTorch Geometric Data objects
graphs = [basic_data[0][0:2]]
dataset_1 = GraphDataset(graphs, pre_paired=use_pre_paired)

data_1, data_2 = dataset_1[0]

print("NODE FEATURES:")

print(get_attr_matrix(graphs[0][0]))
print(get_attr_matrix(graphs[0][1]))

print("---")

print(data_1.x)
print(data_2.x)

print("ADJACENCY:")

print(nx.adjacency_matrix(graphs[0][0]).todense())
print(nx.adjacency_matrix(graphs[0][1]).todense())

print("---")

print(data_1.edge_index)
print(data_2.edge_index)

NODE FEATURES:
[[0 1 0 0 0 0]
 [1 0 0 0 1 0]
 [1 0 0 0 0 1]]
[[0 1 0 0 0 0]
 [1 0 0 0 1 0]
 [1 0 0 0 0 1]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]]
---
tensor([[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1.]])
tensor([[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]])
ADJACENCY:
[[0 1 0]
 [0 0 1]
 [0 1 0]]
[[0 1 0 0 0]
 [0 0 1 1 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]
---
tensor([[0, 1, 2],
        [1, 2, 1]])
tensor([[0, 1, 1, 2, 3],
        [1, 2, 3, 1, 4]])


In [10]:
dataset_1

In [11]:
n_features = dataset[0][0].x.shape[1] if use_pre_paired else dataset[0].x.shape[1]
print(f'Number of features in the dataset: {n_features}')
gnn = GCNFeatureExtractor(in_channels=n_features, out_channels=embedding_size, pooling_strategy=pooling)

# model = torch.nn.Sequential(gnn).to(device) # TODO: see todo in next cell
model = gnn.to(device)

Number of features in the dataset: 6


In [12]:
train(model, dataset, epochs, batch_size, device=device)


# TODO: nn.Sequential expexts a single input, not multiple ones as happens for GNNs (x, edge_index, batch)
# You have to find a way to stack several models, without using nn.Sequential, or to wrap the GNN in a class 
# that handles a single input.

: 